### Este notebook é dedicado a entender o efeito da quarentena e consequente redução do trafego de automóveis na concentração de poluentes no ar da região da Marginal Pinheiros - Ponte dos Remédios em São Paulo.

Antes de tudo, gostaria de agradecer à [CETESB - Companhia Ambiental do Estado de São Paulo](https://cetesb.sp.gov.br/) por coletar e disponibilizar os dado aqui analisados. Os registros podem ser acessados na [página](https://cetesb.sp.gov.br/ar/dados-horarios/).

Foi construido um scrapper para retirar os dados da tabela no HTML da página e obter os registros em formato tabular para mais de um ano.

# A quarentena e medidas de distanciamento sociais contribuiram para a redução na emissão de poluentes na Marginal Pinheiros?

In [1]:
#import libs
import numpy as np
import pandas as pd
import datetime

from bs4 import BeautifulSoup
from mechanize import Browser

from scipy import stats
import math

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

In [3]:
#constants
init_data = datetime.datetime(2019,1,1,0,0)

init_data_pd = pd.to_datetime(init_data)

init_data_str = init_data.strftime('%d-%m-%Y')

data = datetime.datetime(2020,7,1,0,0)

data_str = data.strftime('%d-%m-%Y')

datas = [(data+datetime.timedelta(hours=i)).strftime('%Y-%m-%d-%H-%M') for i in range(1,25)]

date_interval = pd.date_range(start=init_data, end=data)

DICT_WEEKDAY = {
    1: {
        'name': 'monday',
        'pd_freq': 'W-MON'
    }, 
    2: {
        'name': 'tuesday',
        'pd_freq': 'W-TUE'
    },
    3: {
        'name': 'wednesday',
        'pd_freq': 'W-WED'
    },
    4: {
        'name': 'thursday',
        'pd_freq': 'W-THU'
    },
    5: {
        'name': 'friday',
        'pd_freq': 'W-FRI'
    },
    6: {
        'name': 'saturday',
        'pd_freq': 'W-SAT'
    },
    7: {
        'name': 'sunday',
        'pd_freq': 'W-SUN'
    }
}

In [4]:
class scrapData():
    
    def __init__(self, data, est):
        self.url = "https://sistemasinter.cetesb.sp.gov.br/Ar/php/ar_dados_horarios.php"
        self.data = data
        self.est = est
        self.br = Browser()
            
    def loadPage(self):
        self.br.set_handle_robots(False) # ignore robots
        self.br.open(self.url)
        self.br.select_form(name="form1")
        self.br["texData"] = self.data
        self.br["selEst"] = self.est
        self.br.form.action="https://sistemasinter.cetesb.sp.gov.br/Ar/php/ar_dados_horarios_resultado.php"
        self.content = self.br.submit().read()

    def getContent(self):
        return self.content
    
    def writeFile(self, filename):
        with open(filename, "w") as f:
            f.write(str(BeautifulSoup(self.content,"lxml")))

In [5]:
#a primeira data é construida no df41
df4=pd.DataFrame([])
df41 = pd.DataFrame([])
scrapper = scrapData(init_data_pd.strftime('%d-%m-%Y'), ["36/Marg.Tietê-Pte Remédios"])
scrapper.loadPage()
soup = BeautifulSoup(scrapper.content, "lxml")
df4['date'] = [(init_data_pd+datetime.timedelta(hours=i)).strftime('%Y-%m-%d-%H-%M') for i in range(1,25)]

a=[]
for row in soup.find_all('table')[6].find_all(['tr']):
    for cell in row.find_all(['td']):
        a.append(cell.get_text().replace('\t', '').replace('\r\n', ' '))
a=a[4:]
b = list(a[i:i+3] for i in range(0, len(a), 3))
df4 = df4.join(pd.DataFrame(b, columns=['SO2 Média Horária', 'SO2 Média 24h', 'SO2 Indice']))

a=[]
for row in soup.find_all('table')[7].find_all(['tr']):
    for cell in row.find_all(['td']):
        a.append(cell.get_text().replace('\t', '').replace('\r\n', ' '))
a=a[3:]
b = list(a[i:i+2] for i in range(0, len(a), 2))
df4 = df4.join(pd.DataFrame(b, columns=['NO2 Média Horária', 'NO2 Indice']))

a=[]
for row in soup.find_all('table')[9].find_all(['tr']):
    for cell in row.find_all(['td']):
        a.append(cell.get_text().replace('\t', '').replace('\r\n', ' '))
a=a[4:]
b = list(a[i:i+3] for i in range(0, len(a), 3))
df4 = df4.join(pd.DataFrame(b, columns=['CO Média Horária', 'CO Média 24h', 'CO Indice']))

df41 = df41.append(df4)

In [94]:
#df41 recebe como append as demais datas
for dt in date_interval[1:]:
    df4=pd.DataFrame([])
    scrapper = scrapData(dt.strftime('%d-%m-%Y'), ["36/Marg.Tietê-Pte Remédios"])
    scrapper.loadPage()
    soup = BeautifulSoup(scrapper.content, "lxml")
    df4['date'] = [(dt+datetime.timedelta(hours=i)).strftime('%Y-%m-%d-%H-%M') for i in range(1,25)]
    
    a=[]
    for row in soup.find_all('table')[6].find_all(['tr']):
        for cell in row.find_all(['td']):
            a.append(cell.get_text().replace('\t', '').replace('\r\n', ' '))
    a=a[4:]
    b = list(a[i:i+3] for i in range(0, len(a), 3))
    df4 = df4.join(pd.DataFrame(b, columns=['SO2 Média Horária', 'SO2 Média 24h', 'SO2 Indice']))

    a=[]
    for row in soup.find_all('table')[7].find_all(['tr']):
        for cell in row.find_all(['td']):
            a.append(cell.get_text().replace('\t', '').replace('\r\n', ' '))
    a=a[3:]
    b = list(a[i:i+2] for i in range(0, len(a), 2))
    df4 = df4.join(pd.DataFrame(b, columns=['NO2 Média Horária', 'NO2 Indice']))
    
    a=[]
    for row in soup.find_all('table')[9].find_all(['tr']):
        for cell in row.find_all(['td']):
            a.append(cell.get_text().replace('\t', '').replace('\r\n', ' '))
    a=a[4:]
    b = list(a[i:i+3] for i in range(0, len(a), 3))
    df4 = df4.join(pd.DataFrame(b, columns=['CO Média Horária', 'CO Média 24h', 'CO Indice']))
    
    df41 = df41.append(df4)
    
    
    

In [97]:
df41.head()

,date,SO2 Média Horária,SO2 Média 24h,SO2 Indice,NO2 Média Horária,NO2 Indice,CO Média Horária,CO Média 24h,CO Indice
0,2019-01-01-01-00,2,2,4,NaN,NaN,0.4,"0,3",1
1,2019-01-01-02-00,1,1,2,13,3,0.4,"0,3",1
2,2019-01-01-03-00,6,2,4,17,3,0.4,"0,3",1
3,2019-01-01-04-00,NaN,2,4,20,4,0.4,"0,3",2
4,2019-01-01-05-00,8,2,4,20,4,NaN,"0,4",2


In [137]:
df41 = df41.sort_values(by='date')

In [138]:
#save dataset
df41.to_csv('qualidade_do_ar_{}_{}_marg_tiete.csv'.format(init_data_str, data_str), sep=';', index=False)

In [139]:
#Tratando vazios
df41 = df41.replace('--', np.nan)

df42 = df41.copy()

In [140]:
df42 = df42.reset_index().drop(columns='index')

In [141]:
# Uma das colunas veio com o decimal trocado
df42['CO Média 24h'] = [x.replace(',', '.') for x in df42['CO Média 24h'].astype('str')]

In [142]:
#Convertendo os tipos
for col in df42.columns[1:]:
    df42[col] = df42[col].astype('float')

In [143]:
#Tratamento das datas

df42['date'] = pd.to_datetime(df42['date'])
df42['year'] = df42['date'].dt.year
df42['month'] = df42['date'].dt.month
df42['day'] = df42['date'].dt.day
df42['hour'] = df42['date'].dt.hour

for index, row in df42.iterrows():
    df42.loc[index, 'weekday'] = DICT_WEEKDAY[pd.to_datetime(row['date']).isocalendar()[2]]['name']
    
    
#o delta está em semanas para contornar sazonalidade semanais    
df42['date_last_year'] = df42['date'] - datetime.timedelta(weeks=52)

In [144]:
index = (df42['month'].isin([3,4,5])) & (df42['hour']==4)

for col in df42.columns[1:4]:
    print('Porcentagem de valores vazios da regiao de março a maio as 4 horas: ', col ,100*df42[index][col].isna().sum()/df42[index][col].shape[0])

Porcentagem de valores vazios da regiao de março a maio as 4 horas:  SO2 Média Horária 96.19565217391305
Porcentagem de valores vazios da regiao de março a maio as 4 horas:  SO2 Média 24h 0.5434782608695652
Porcentagem de valores vazios da regiao de março a maio as 4 horas:  SO2 Indice 0.5434782608695652


In [145]:
index = (df42['month'].isin([3,4,5])) & (df42['hour']==1)

for col in df42.columns[4:6]:
    print('Porcentagem de valores vazios da regiao de março a maio a 1 hora: ', col ,100*df42[index][col].isna().sum()/df42[index][col].shape[0])

Porcentagem de valores vazios da regiao de março a maio a 1 hora:  NO2 Média Horária 97.28260869565217
Porcentagem de valores vazios da regiao de março a maio a 1 hora:  NO2 Indice 97.28260869565217


In [146]:
index = (df42['month'].isin([3,4,5])) & (df42['hour']==5)

for col in df42.columns[6:9]:
    print('Porcentagem de valores vazios da regiao de março a maio em 2019 as 5 horas: ', col ,100*df42[index][col].isna().sum()/df42[index][col].shape[0])

Porcentagem de valores vazios da regiao de março a maio em 2019 as 5 horas:  CO Média Horária 99.45652173913044
Porcentagem de valores vazios da regiao de março a maio em 2019 as 5 horas:  CO Média 24h 2.717391304347826
Porcentagem de valores vazios da regiao de março a maio em 2019 as 5 horas:  CO Indice 2.717391304347826


Nota-se que as medidas estão OK, sendo que existe um horário diariamente em que a concentração de poluente não é coletada. 

Para o NO2 não é coletado na hora 01. Para CO na hora 05 e para o SO2 na hora 04.

#### Série temporal da concentração de cada um dos poluentes

In [147]:
def plotMediaHoraria(df, year, month, pol):
    mediaHorariaPol = pol + ' Média Horária'
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=df[(df['month'].isin(month)) & (df['year']==year)]['date'],
                             y=df[(df['month'].isin(month)) & (df['year']==year)][mediaHorariaPol],
                             mode='lines'))
    fig.update_layout(title='{} Média Horária {}'.format(pol, year))
    fig.update_layout(height=400)
    fig.show()

    fig=go.Figure()
    fig.add_trace(go.Scatter(x=df[(df['month'].isin(month)) & (df['year']==year+1)]['date'],
                             y=df[(df['month'].isin(month)) & (df['year']==year+1)][mediaHorariaPol],
                             mode='lines'))
    fig.update_layout(title='{} Média Horária {}'.format(pol, year+1))
    fig.update_layout(height=400)
    fig.show()

In [149]:
plotMediaHoraria(df42[df42['hour']!=1], 2019, [3,4,5], 'NO2')
plotMediaHoraria(df42[df42['hour']!=5], 2019, [3,4,5], 'CO')
plotMediaHoraria(df42[df42['hour']!=4], 2019, [3,4,5], 'SO2')

### Os dados são consistentes, sem "buracos" além daqueles dos horarios que não são medidos.
Portanto, será feito um preenchimento dos dados faltantes com o ultimo valor válido na serie temporal.

In [150]:
# 'ffill' para ultimo valor válido
df42 = df42.fillna(method='ffill')

#### Comparativo das series temporais dos dois anos após limpeza do dataset

In [151]:
fig = make_subplots(rows=3, cols=1, column_widths=[2],
                    subplot_titles=("CO Média Horária", "NO2 Média Horária", "SO2 Média Horária"))

indexLastYear = (df42['date'].dt.year == 2019) & (df42['date'].dt.month.isin([3,4,5])) #& (df42['date'].dt.hour.isin([14,15,16,17]))# & (df42['weekday'].isin(['sunday', 'saturday'])==False) 
index = (df42['date'].dt.year == 2020) & (df42['date'].dt.month.isin([3,4,5])) #& (df42['date'].dt.hour.isin([14,15,16,17]))# & (df42['weekday'].isin(['sunday', 'saturday'])==False) 

# fig = go.Figure()
fig.add_trace(go.Scatter(x=df42[indexLastYear]['date']+datetime.timedelta(weeks=52), 
                         y=df42[indexLastYear]['CO Média Horária'],
                    mode='lines',
                    line=dict(color='firebrick'),
                    name='2019'), 
              row=1, col=1)

fig.add_trace(go.Scatter(x=df42[index]['date'], 
                         y=df42[index]['CO Média Horária'],
                         mode='lines', line=dict(color='royalblue'), 
                         name='2020'), 
              row=1, col=1)

fig.add_trace(go.Scatter(x=df42[indexLastYear]['date']+datetime.timedelta(weeks=52), 
                         y=df42[indexLastYear]['NO2 Média Horária'],
                         mode='lines',
                         line=dict(color='firebrick'),
                         showlegend=False,), 
              row=2, col=1)

fig.add_trace(go.Scatter(x=df42[index]['date'], 
                         y=df42[index]['NO2 Média Horária'],
                        mode='lines',
                        line=dict(color='royalblue'),
                        showlegend=False,), 
              row=2, col=1)

fig.add_trace(go.Scatter(x=df42[indexLastYear]['date']+datetime.timedelta(weeks=52), 
                         y=df42[indexLastYear]['SO2 Média Horária'],
                         mode='lines',
                         line=dict(color='firebrick'),
                         showlegend=False,), 
              row=3, col=1)

fig.add_trace(go.Scatter(x=df42[index]['date'], 
                         y=df42[index]['SO2 Média Horária'],
                         mode='lines',
                         line=dict(color='royalblue'),
                         showlegend=False,), 
             row=3, col=1)

fig.update_layout(height=900)
fig.show()

In [152]:
df42meanDia = df42.groupby(['year', 'month', 'day', 'weekday']).agg({'date': 'first',
                                                                    'CO Média Horária': 'mean',
                                                                    'NO2 Média Horária': 'mean',
                                                                    'SO2 Média Horária': 'mean',}).reset_index()

In [153]:
def plotComparativosMediaDiaria():
    indexLastYear = (df42meanDia['date'].dt.year == 2019) & (df42meanDia['date'].dt.month.isin([3,4,5]))
    index = (df42meanDia['date'].dt.year == 2020) & (df42meanDia['date'].dt.month.isin([3,4,5]))

    fig = make_subplots(rows=3, cols=1, column_widths=[2],
                        subplot_titles=("CO Média Diária", "NO2 Média Diária", "SO2 Média Diária"))

    fig.add_trace(go.Scatter(x=df42meanDia[indexLastYear]['date']+datetime.timedelta(weeks=52), 
                             y=df42meanDia[indexLastYear]['CO Média Horária'],
                        mode='lines',
                        line=dict(color='firebrick'),
                        name='2019'), 
                  row=1, col=1)

    fig.add_trace(go.Scatter(x=df42meanDia[index]['date'], 
                             y=df42meanDia[index]['CO Média Horária'],
                             mode='lines', line=dict(color='royalblue'), 
                             name='2020'), 
                  row=1, col=1)

    fig.add_trace(go.Scatter(x=df42meanDia[indexLastYear]['date']+datetime.timedelta(weeks=52), 
                             y=df42meanDia[indexLastYear]['NO2 Média Horária'],
                             mode='lines',
                             line=dict(color='firebrick'),
                             showlegend=False,), 
                  row=2, col=1)

    fig.add_trace(go.Scatter(x=df42meanDia[index]['date'], 
                             y=df42meanDia[index]['NO2 Média Horária'],
                            mode='lines',
                            line=dict(color='royalblue'),
                            showlegend=False,), 
                  row=2, col=1)

    fig.add_trace(go.Scatter(x=df42meanDia[indexLastYear]['date']+datetime.timedelta(weeks=52), 
                             y=df42meanDia[indexLastYear]['SO2 Média Horária'],
                             mode='lines',
                             line=dict(color='firebrick'),
                             showlegend=False,), 
                  row=3, col=1)

    fig.add_trace(go.Scatter(x=df42meanDia[index]['date'], 
                             y=df42meanDia[index]['SO2 Média Horária'],
                             mode='lines',
                             line=dict(color='royalblue'),
                             showlegend=False,), 
                 row=3, col=1)

    fig.update_layout(height=1000)
    fig.show()

##### É notável que os valores de concentração do ano de 2020, curva azul, estão abaixo dos valores de 2019, curva vermelha

In [154]:
plotComparativosMediaDiaria()

In [155]:
def plotComparativosMediaDiariaHist():
    indexLastYear = (df42['date'].dt.year == 2019) & (df42['date'].dt.month.isin([3,4,5]))
    index = (df42['date'].dt.year == 2020) & (df42['date'].dt.month.isin([3,4,5]))

    fig = make_subplots(rows=3, cols=1, column_widths=[2],
                        subplot_titles=("CO Média Horária", "NO2 Média Horária", "SO2 Média Horária"),
                        shared_xaxes=False)

    fig.add_trace(go.Histogram(x=df42[index]['CO Média Horária'], 
                               nbinsx=30, name='2020',
                               marker_color='blue'),
                 row=1, col=1)
    fig.add_trace(go.Histogram(x=df42[indexLastYear]['CO Média Horária'],
                               nbinsx=30, name='2019',
                               marker_color='red'),
                 row=1, col=1)

    fig.add_trace(go.Histogram(x=df42[index]['NO2 Média Horária'], 
                               nbinsx=100, name='2020',
                               marker_color='blue',
                               showlegend=False),
                 row=2, col=1)
    fig.add_trace(go.Histogram(x=df42[indexLastYear]['NO2 Média Horária'],
                               nbinsx=100, name='2019',
                               marker_color='red',
                               showlegend=False),                           
                 row=2, col=1)

    fig.add_trace(go.Histogram(x=df42[index]['SO2 Média Horária'], 
                               nbinsx=20, name='2020',
                               marker_color='blue',
                               showlegend=False),
                 row=3, col=1)
    fig.add_trace(go.Histogram(x=df42[indexLastYear]['SO2 Média Horária'],
                               nbinsx=20, name='2019',
                               marker_color='red',
                               showlegend=False),                           
                 row=3, col=1)
    fig.update_traces(opacity=0.5)

    # Overlay both histograms
    fig.update_layout(barmode='overlay')
    fig.update_layout(height=1000,
                      bargap=0, # gap between bars of adjacent location coordinates
                      bargroupgap=0) # gap between bars of the same location coordinates)
    fig.show()

In [156]:
plotComparativosMediaDiariaHist()

#### Percebe-se que o histograma de 2020 para CO e NO2 está mais deslocado para a esquerda quando comparamos os dois anos. O que significa registros menores de concentração de poluentes nos meses de março a maio

In [83]:
indexLastYear = (df42['date'].dt.year == 2019) & (df42['date'].dt.month.isin([3,4,5]))# & (df42['date'].dt.hour.isin([19,20,21]))# & (df42['weekday'].isin(['sunday', 'saturday'])==False) 
index = (df42['date'].dt.year == 2020) & (df42['date'].dt.month.isin([3,4,5]))# & (df42['date'].dt.hour.isin([19,20,21]))# & (df42['weekday'].isin(['sunday', 'saturday'])==False) 

x1 = x=df42[index]['NO2 Média Horária']
x2 = x=df42[indexLastYear]['NO2 Média Horária']

group_labels = ['2020', '2019']

colors = ['royalblue', 'firebrick']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot([x1, x2], group_labels, bin_size=3,
                         curve_type='normal', # override default 'kde'
                         colors=colors)

# Add title
fig.update_layout(title_text='Concentração de NO2')
fig.show()

In [84]:
indexLastYear = (df42['date'].dt.year == 2019) & (df42['date'].dt.month.isin([3,4,5]))
index = (df42['date'].dt.year == 2020) & (df42['date'].dt.month.isin([3,4,5]))

x1 = x=df42[index]['CO Média Horária']
x2 = x=df42[indexLastYear]['CO Média Horária']

group_labels = ['2020', '2019']

colors = ['royalblue', 'firebrick']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot([x1, x2], group_labels, bin_size=.1,
                         curve_type='normal', # override default 'kde'
                         colors=colors)

# Add title
fig.update_layout(title_text='Concentração de CO')
fig.show()

In [85]:
#avaliação da redução de concentração dos poluentes em 2019 e 2020

indexLastYear = (df42['date'].dt.year == 2019) & (df42['date'].dt.month.isin([2,3,4,5,6]))
index = (df42['date'].dt.year == 2020) & (df42['date'].dt.month.isin([2,3,4,5,6]))

razaodfCO = df42[indexLastYear][['date', 'month', 'CO Média Horária']].rename(columns={'CO Média Horária': 'CO Média Horária 2019'})\
.merge(df42[index][['date_last_year', 'CO Média Horária']].rename(columns={'CO Média Horária': 'CO Média Horária 2020'}), left_on='date', right_on='date_last_year')

razaodfCO['razao_CO'] = 1 - (razaodfCO['CO Média Horária 2020']/razaodfCO['CO Média Horária 2019'])
razaodfCO['razao_CO'] = razaodfCO['razao_CO'].astype('float')

razaodfNO2 = df42[indexLastYear][['date', 'month', 'NO2 Média Horária']].rename(columns={'NO2 Média Horária': 'NO2 Média Horária 2019'})\
.merge(df42[index][['date_last_year', 'NO2 Média Horária']].rename(columns={'NO2 Média Horária': 'NO2 Média Horária 2020'}), left_on='date', right_on='date_last_year')

razaodfNO2['razao_NO2'] = 1 - (razaodfNO2['NO2 Média Horária 2020']/razaodfNO2['NO2 Média Horária 2019'])
razaodfNO2['razao_NO2'] = razaodfNO2['razao_NO2'].astype('float')

razaodfSO2 = df42[indexLastYear][['date', 'month', 'SO2 Média Horária']].rename(columns={'SO2 Média Horária': 'SO2 Média Horária 2019'})\
.merge(df42[index][['date_last_year', 'SO2 Média Horária']].rename(columns={'SO2 Média Horária': 'SO2 Média Horária 2020'}), left_on='date', right_on='date_last_year')

razaodfSO2['razao_SO2'] = 1 - (razaodfSO2['SO2 Média Horária 2020']/razaodfSO2['SO2 Média Horária 2019'])
razaodfSO2['razao_SO2'] = razaodfSO2['razao_SO2'].astype('float')

In [86]:
rz = razaodfCO.merge(razaodfNO2, on=['date', 'date_last_year', 'month'])\
.merge(razaodfSO2, on=['date', 'date_last_year', 'month'])

In [87]:
notNan = ((rz['CO Média Horária 2019']!=0) &
         (rz['NO2 Média Horária 2019']!=0) & 
         (rz['SO2 Média Horária 2019']!=0))

In [88]:
rz_month = rz[notNan].groupby('month').agg({'razao_CO': 'mean',
                                            'razao_NO2': 'mean',
                                            'razao_SO2': 'mean'}).reset_index()

In [89]:
rz_month

,month,razao_CO,razao_NO2,razao_SO2
0,2,-0.013892,-0.066163,-0.780358
1,3,0.148349,0.087615,-0.293266
2,4,0.268712,0.287222,-0.205170
3,5,0.076244,0.019588,-0.456131
4,6,-0.355701,-0.015420,-0.136929


### É notavel a diferença entre a média da razão de CO entre os registros de diferentes meses. 
Quando a quarentena estava começando no meio de mês de março, já houve efeito na concentração média mensal de CO registrada
Essa redução atinge seu pico em abril com registros, em média, 26% menores. Em maio e junho, o movimento retorna e os valores sobem.


In [90]:
colors = ['royalblue',] * 5
colors[2] = 'blue'
colors[4] = 'firebrick'

fig = go.Figure(data=[
    go.Bar(name='Concentração de CO', 
           x=['fev', 'mar', 'abr', 'mai', 'jun'], 
           y=rz_month['razao_CO'],
           text=[format(100*x, '.2f') for x in rz_month['razao_CO']],
           textposition='inside',
          marker_color=colors),
])

# Change the bar mode
fig.update_layout(barmode='group', title='Razão na concentração de CO - Comparativo 2019 vs 2020')
fig.show()

In [163]:
colors = ['royalblue',] * 5
colors[0] = 'firebrick'
colors[2] = 'blue'
colors[4] = 'firebrick'

fig = go.Figure(data=[
    go.Bar(name='Concentração de NO2', 
           x=['fev', 'mar', 'abr', 'mai', 'jun'], 
           y=rz_month['razao_NO2'],
           text=[format(100*x, '.2f') for x in rz_month['razao_NO2']],
           textposition='outside',
          marker_color=colors),
])
# Change the bar mode
fig.update_layout(barmode='group', title='Razão na concentração de NO2 - Comparativo 2019 vs 2020')
fig.show()

In [162]:
colors = ['royalblue',] * 5
colors[0] = 'firebrick'
colors[2] = 'blue'
colors[4] = 'firebrick'

fig = go.Figure(data=[
    go.Bar(name='Concentração de SO2', 
           x=['fev', 'mar', 'abr', 'mai', 'jun'], 
           y=rz_month['razao_SO2'],
           text=[format(100*x, '.2f') for x in rz_month['razao_SO2']],
           textposition='outside',
          marker_color=colors),
])
# Change the bar mode
fig.update_layout(barmode='group', title='Razão na concentração de NO - Comparativo 2019 vs 2020')
fig.show()

In [92]:
sample_size = rz[notNan].shape[0]
sample = np.random.choice(a= rz[notNan & (rz['month']==4)]['razao_CO'], size = sample_size)
sample_mean = sample.mean()

z_critical = stats.norm.ppf(q = 0.95)  # Get the z-critical value*

print("z-critical value:")              # Check the z-critical value
print(z_critical)                        

pop_stdev = rz[notNan & (rz['month']==4)]['razao_CO'].std()  # Get the population standard deviation

margin_of_error = z_critical * (pop_stdev/math.sqrt(sample_size))

confidence_interval = (sample_mean - margin_of_error,
                       sample_mean + margin_of_error)  

print("Confidence interval:")
print(confidence_interval)

z-critical value:
1.6448536269514722
Confidence interval:
(0.2511592427076715, 0.27561239606950744)


No mês de abril, houve uma redução de 25.5% a 28% na concentração de CO com 95% de intervalo de confiança.

A curva normal de distribuição dessa razão deslocada para a direita indica claramente a redução.

In [93]:
x1 = rz[notNan  & (rz['month']==4)]['razao_CO']

group_labels = ['2020']

colors = ['royalblue']

fig = ff.create_distplot([x1], group_labels, bin_size=.1,
                         curve_type='normal', # override default 'kde'
                         colors=colors)

# Add title
fig.update_layout(title_text='Concentração de CO em Abril')
fig.show()